<h2  style="color:green">Lab: Access Db2 database on Cloud using Python</b></h2>

<h3 style="color:blue">Introduction<strong></h3>

In this section we will cover following topics,

    1. Create a table
    2. Insert data into the table
    3. QUery data from the table
    4. Retrieve the result set and store it into a pandas dataframe


<h5>[a] Import Python library to perform database operations </h5>

In [1]:
import ibm_db

<h5>[b] Get database connection credentials </h5>

To create a successful connection between our notebook and the Db2 IBM data server, we need to pass following credentials:

    1. Driver Name
    2. Database name
    3. Host DNS name or IP address
    4. Host port
    5. Connection protocol
    6. User ID (or username)
    7. User Password

Follow this link <a href='https://d18ky98rnyall9.cloudfront.net/hkYJxA4gEem5_xLqNrIdUA_86861fb00e2011e99ff16dafffcf4ae2_Wk3-LAB-0---v5---Create-Db2-Service-Credentials.pdf?Expires=1563408000&Signature=TgDnJ-9XfGxqZqwhRkwibbhdxbdG68FWy1EhCdR0DTIuWGlGBAVn6bTjT8W-oZyQvUgVXl7gsgQXyuqRsnz9ZyPsYaurVtdPjMiJ33P2idikKK5oI-cyAzaAtaUk~6Yuz7yDs8RwcCT6XXF7RzQGYP6vTqE1JM9mOpWYdhxV-y4_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'>Create Credentials</a> link to retrieve connection credentials for your Db2 IBM data server.

In [2]:
dsn_hostname = "****"
dsn_uid = "****"
dsn_pwd = "****"

dsn_driver = "****"
dsn_database = "****"
dsn_port = "****"
dsn_protocol = "****"

<h5>[c] Establish database connection </h5>

In [3]:
# Create a dsn connection string
# it will be used while establishing connection with IBM Data Server
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

In [4]:
# Establish a database connection

try:
    db_connection = ibm_db.connect(dsn,"","")
    print("Connected to database: ", dsn_database,"as user: ", dsn_uid, "on host: ",dsn_hostname)
    
except:
    print("Unable to connect: ", ibm_db.conn_errormsg())

Connected to database:  BLUDB as user:  hrt90440 on host:  dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net


<h5>[1] Create a table in the database </h5>

In this step we will create a table in the database with following details:

<img src = "https://ibm.box.com/shared/static/ztd2cn4xkdoj5erlk4hhng39kbp63s1h.jpg" align="center">


<span> We will use <code>exec_immediate()</code> method to query the database</span>

In [5]:
# Drop table 'INSTRUCTOR' if it already created

dropTableQuery = "drop table INSTRUCTOR;"

dropStmt = ibm_db.exec_immediate(db_connection, dropTableQuery)

# If you got an error staring <database_name>.INSTRUCTOR is an undefined name
# It is because INSTRUCTOR table is not created previously, 
# and we tried to delete a non-sxisting table from the database

In [6]:
# Create a table INSTRUCTOR and pass the column names and their attributes 
# as given in the above diagram

createTableQuery = "create table INSTRUCTOR\
                    (id INTEGER PRIMARY KEY NOT NULL,\
                    fname VARCHAR(50),\
                    lname VARCHAR(50),\
                    city VARCHAR(50),\
                    code CHAR(4));"


createTableStmt = ibm_db.exec_immediate(db_connection, createTableQuery)

<h5>[2] Insert data into the table </h5>

In this step we will insert some rows of data into the table. 

The INSTRUCTOR table we created in the previous step contains 3 rows of data:

<img src="https://ibm.box.com/shared/static/j5yjassxefrjknivfpekj7698dqe4d8i.jpg" align="center">


In [7]:
# Insert data query
insertQuery = "insert into INSTRUCTOR(id, fname, lname, city, code)\
               VALUES\
               (1, 'Rav', 'Ahuja','Toronto','CA'),\
               (2, 'Raul', 'Chong','Markham','CA'),\
               (3, 'Hima', 'Vasudevan','Chicago','US');"
               
# Execute the query
insertStmt = ibm_db.exec_immediate(db_connection, insertQuery)

<h5>[3] Query data in the table</h5>

We will retrive all the data from INSTRUCTOR table

In [8]:
# Select query to retrieve all the data
selectQuery = "select * from INSTRUCTOR;"

# Execute the statement
selectStmt = ibm_db.exec_immediate(db_connection, selectQuery)

# Fetch results and print Id and first name of each record
while ibm_db.fetch_both(selectStmt):
    print("ID: ", ibm_db.result(selectStmt, 0), "\t\tFirst Name: ",ibm_db.result(selectStmt, "FNAME"))

ID:  1 		First Name:  Rav
ID:  2 		First Name:  Raul
ID:  3 		First Name:  Hima


In [9]:
# Lets update the city of Rav Ahuja from Torono to Moosetown

updateQuery = "update INSTRUCTOR\
               set CITY='Moosetown'\
               where FNAME = 'Rav';"

updateStmt = ibm_db.exec_immediate(db_connection,updateQuery)

In [10]:
# Select query to retrieve all the data
selectQuery = "select * from INSTRUCTOR;"

# Execute the statement
selectStmt = ibm_db.exec_immediate(db_connection, selectQuery)

# Fetch results and print Id and first name of each record
while ibm_db.fetch_both(selectStmt):
    print("ID: ", ibm_db.result(selectStmt, 0),\
          "\t\tFirst Name: ",ibm_db.result(selectStmt, "FNAME"),\
         "\t\tCity:", ibm_db.result(selectStmt,"CITY"))

ID:  1 		First Name:  Rav 		City: Moosetown
ID:  2 		First Name:  Raul 		City: Markham
ID:  3 		First Name:  Hima 		City: Chicago


<h5>[4] Retrive data into Pandas</h5>

In [11]:
import pandas as pd
import ibm_db_dbi

In [12]:
# Connection for pandas
pConnection = ibm_db_dbi.Connection(db_connection)

In [13]:
# Query statement to retrieve all rows from INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR;"

# we use inbuilt read_sql function of pandas library to retrieve data
pDataFrame = pd.read_sql(selectQuery, pConnection)

# print retrieved table details
pDataFrame

,ID,FNAME,LNAME,CITY,CODE
0,1,Rav,Ahuja,Moosetown,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US
